In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import ast, os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tensorflow.keras.preprocessing.sequence import pad_sequences
import seaborn as sns
# from keras.models import model_from_yaml

In [6]:
df = pd.read_csv('../Data/Flujo1.csv', sep=";")
df['Respuesta'] = df['Respuesta'].apply(ast.literal_eval
def wordsToNumbers(tokens, vocabulary):
    number_array = []
    for i in tokens:
        number_array.append(vocabulary.index(i)+1)
    return np.asarray(number_array)    

In [7]:
#Generamos un vocabulario de palabras
vocabulary = []
df['Respuesta'].apply(vocabulary.append)
vocabulary = [item for sublist in vocabulary for item in sublist]
vocabulary = list(set(vocabulary))

# Se Construye Dataframe solo de Social Externo
df = df[df.Area == 'Social Externo']
# Lista de dimensiones se pasa a numeros
list_dimensions = df.Dimension.unique().tolist()
num_classes = len(list_dimensions)
df['_dimension'] = df['Dimension'].apply(lambda dimension, list_dimensions: list_dimensions.index(dimension), list_dimensions = list_dimensions)

df['NumRespuesta'] = df.Respuesta.apply (wordsToNumbers, vocabulary = vocabulary)
# Rellenamos las matrices con 0 para que todas tengan el mismo tamaño
X_completo = pad_sequences(df.NumRespuesta, maxlen=10, dtype='object', padding='post', value = 0)
# Pasamos las dimensiones a np array
y_completo = np.array(df['_dimension'])
X, test_x, y, test_y = train_test_split(X_completo, y_completo, test_size = 0.2, random_state = 0)
resultados = []

D:\ProgramasVarios\Anaconda\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
D:\ProgramasVarios\Anaconda\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [26]:
##Probando combinaciones de redes ---- 
resultados = []

vocab_len= len(vocabulary)+1
num_classes = len(list_dimensions)
batch_size = 60
ouput_dim_embedding_range = [50,100,200,300]
activationFunctionsLSTM = ['sigmoid', 'relu','softmax']
lstm_unit = [50, 100, 150]
dropuotLSTM = [0.3,0.5,0.7]
recu_dropuotLSTM = [0.3]
activationFunctionsDense = ['sigmoid', 'relu','softmax']
optimizersNN = ['rmsprop','adam','adadelta','sgd']
# epochs =  [5,6,7,8,9,10,13,15,17,20,25,30,40,50,60,80]
epoch = 12
for ouput_dim_embedding in ouput_dim_embedding_range:
    for unit in lstm_unit:
        for activationFunctionLSTM in activationFunctionsLSTM:
            for drop in dropuotLSTM:
                for recu_drop in recu_dropuotLSTM:
                    for activationFunctionDense in activationFunctionsDense:
                        for optimizerNN in optimizersNN:
                            model = tf.keras.Sequential()
                            model.add(tf.keras.layers.Embedding(input_dim=vocab_len, 
                                                                output_dim = ouput_dim_embedding, 
                                                                input_shape=(X.shape[1],)
                                                               )
                                     )

                            model.add(tf.keras.layers.LSTM(units = unit, 
                                                           activation=activationFunctionLSTM, 
                                                           dropout=drop, 
                                                           recurrent_dropout=recu_drop
                                                          )
                                     )
                            model.add(tf.keras.layers.Dense(units = num_classes, 
                                                            activation=activationFunctionLSTM
                                                           )
                                     )
                            model.compile(optimizer=optimizerNN,
                                          loss='sparse_categorical_crossentropy',
                                          metrics=['sparse_categorical_accuracy']
                                         )
                            history = model.fit(
                                tf.convert_to_tensor(list(X)),
                                tf.convert_to_tensor(list(y)),
                                epochs = epoch,
                                batch_size = batch_size,
                                workers = 3,
                                use_multiprocessing = True,
                                verbose = 0
                            )
                            test_loss, test_acurracy = model.evaluate(
                                                                tf.convert_to_tensor(list(test_x)),
                                                                tf.convert_to_tensor(list(test_y))
                            )
                            predictions = model.predict_classes(tf.convert_to_tensor(list(test_x))
                            confusion = tf.math.confusion_matrix(
                                                labels=tf.convert_to_tensor(list(test_y)),
                                                predictions= predictions, dtype=tf.dtypes.int32, num_classes=num_classes
                            )
                            _ = {
                                 'ouput_dim_embedding':ouput_dim_embedding,
                                 'actFuncLSTM':activationFunctionLSTM, 
                                 'dropuotLSTM':drop,
                                 'recu_drop': recu_drop,
                                 'activationFunctionDense':activationFunctionDense,
                                 'optimizersNN':optimizersNN,
                                 'accuraccy': test_acurracy,
                                 'loss':test_loss,
                                 'conf_matrix':confusion
                                }
                            if(_['accuraccy'] > 0.7):
                                print(_)
                            resultados.append(_)


14008/1 [===============================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type int).

In [13]:
vocab_len= len(vocabulary)+1
num_classes = len(list_dimensions) #9
ouput_dim_embedding = 90
lstm_units = 60
ouputs_dense = num_classes
epoch = 12
batch_size = 60
optimizer = 'rmsprop'

model = tf.keras.Sequential()
#Se agrega capa embedding que hace w2v
model.add(tf.keras.layers.Embedding(input_dim=vocab_len, output_dim = ouput_dim_embedding, input_shape=(X_completo.shape[1],)))
model.add(tf.keras.layers.LSTM(units = lstm_units, activation='sigmoid',dropout=0.3, recurrent_dropout=0.3))
model.add(tf.keras.layers.Dense(units = ouputs_dense, activation='sigmoid'))
model.compile(optimizer = optimizer, loss = 'sparse_categorical_crossentropy', metrics = ['sparse_categorical_accuracy'])
history = model.fit(X, y, epochs = epoch, batch_size = batch_size)
print()
print('Resultados con datos de testeo ')
print()
test_loss, test_acurracy = model.evaluate(test_x, test_y)
print()
print('Data Test accuracy: {}'.format(test_acurracy))
resultados.append(test_acurracy)
predictions = model.predict_classes(test_x)
#Matriz de confusion
confusion = tf.confusion_matrix(labels=test_y, predictions= predictions, dtype=tf.dtypes.int32, num_classes=14)
print(confusion.eval(session=tf.Session()))
resultados

Train on 55570 samples
Epoch 1/12
55570/55570 [==============================] - 18s 317us/sample - loss: 2.2780 - sparse_categorical_accuracy: 0.2490
Epoch 2/12
55570/55570 [==============================] - 17s 305us/sample - loss: 1.8365 - sparse_categorical_accuracy: 0.3872
Epoch 3/12
55570/55570 [==============================] - 17s 305us/sample - loss: 1.5873 - sparse_categorical_accuracy: 0.4936
Epoch 4/12
55570/55570 [==============================] - 17s 299us/sample - loss: 1.4639 - sparse_categorical_accuracy: 0.5383
Epoch 5/12
55570/55570 [==============================] - 17s 298us/sample - loss: 1.3761 - sparse_categorical_accuracy: 0.5644
Epoch 6/12
55570/55570 [==============================] - 16s 295us/sample - loss: 1.3130 - sparse_categorical_accuracy: 0.5811
Epoch 7/12
55570/55570 [==============================] - 16s 294us/sample - loss: 1.2619 - sparse_categorical_accuracy: 0.5963
Epoch 8/12
55570/55570 [==============================] - 16s 293us/sample - loss

[0.5627294,
 0.54775786,
 0.5617937,
 0.5631613,
 0.5603541,
 0.5408479,
 0.557547,
 0.55833876,
 0.56301737,
 0.5650327]

In [16]:
model_yaml = model.to_yaml()
with open("drive/My Drive/Tesis/RNN/SEXT/modelSEXT.yaml", "w") as yaml_file:
    yaml_file.write(model_yaml)
# serialize weights to HDF5
model.save_weights("drive/My Drive/Tesis/RNN/SEXT/modelSEXT.h5")
print("Saved model to disk")

Saved model to disk


In [0]:
kf = KFold(n_splits = 10, shuffle = True, random_state = 2)
resultados = []
contador = 1
for valores_entrenamiento, valores_testeo in kf.split(X):
    model = tf.keras.Sequential()
    #Se agrega capa embedding que hace w2v
    model.add(tf.keras.layers.Embedding(input_dim=len(vocabulary) + 1, output_dim = 100, input_shape=(X_completo.shape[1],)))
    model.add(tf.keras.layers.LSTM(units=90, activation='sigmoid'))
    model.add(tf.keras.layers.Dense(units=14, activation='sigmoid'))
    model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])
    history = model.fit(X_completo[valores_entrenamiento], y_completo[valores_entrenamiento], epochs = 7, batch_size=15)
    print()
    print(contador, '-Resultados con datos de testeo ')
    print()
    test_loss, test_acurracy = model.evaluate(X[valores_testeo], y[valores_testeo])
    print()
    print(contador, '-Data Test accuracy: {}'.format(test_acurracy))
    resultados.append(test_acurracy)
    print()
    predictions = model.predict_classes(X[valores_testeo])
    #Matriz de confusion
    confusion = tf.confusion_matrix(labels=y[valores_testeo], predictions= predictions, dtype=tf.dtypes.int32, num_classes=14)
    print(contador, '-Matriz de confusion')
    print(confusion.eval(session=tf.Session()))
    print()
    contador = contador + 1

In [0]:
valores_testeo

array([    0,     2,    30, ..., 55558, 55560, 55567])